#undersampling

In [ ]:
min_samples = 1381 # we have these many SCIENCE articles and SCIENCE is our minority class
df_business = df[df.category=="BUSINESS"].sample(min_samples, random_state=2022)
df_sports = df[df.category=="SPORTS"].sample(min_samples, random_state=2022)
df_crime = df[df.category=="CRIME"].sample(min_samples, random_state=2022)
df_science = df[df.category=="SCIENCE"].sample(min_samples, random_state=2022)
df_balanced = pd.concat([df_business,df_sports,df_crime,df_science],axis=0)
df_balanced.category.value_counts()

#BoW n-grams

In [74]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(ngram_range=(1,2))
v.fit(["Thor Hathodawala is looking for a job"])
v.vocabulary_

{'thor': 9,
 'hathodawala': 2,
 'is': 4,
 'looking': 7,
 'for': 0,
 'job': 6,
 'thor hathodawala': 10,
 'hathodawala is': 3,
 'is looking': 5,
 'looking for': 8,
 'for job': 1}

In [75]:
v.transform(["Thor eat pizza is looking"]).toarray()

array([[0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0]])

In [ ]:
import pandas as pd
df = pd.read_json('news_dataset.json')
df.category.value_counts()


In [ ]:
min_samples = 1381 # we have these many SCIENCE articles and SCIENCE is our minority class
df_business = df[df.category=="BUSINESS"].sample(min_samples, random_state=2022)
df_sports = df[df.category=="SPORTS"].sample(min_samples, random_state=2022)
df_crime = df[df.category=="CRIME"].sample(min_samples, random_state=2022)
df_science = df[df.category=="SCIENCE"].sample(min_samples, random_state=2022)
df_balanced = pd.concat([df_business,df_sports,df_crime,df_science],axis=0)
df_balanced.category.value_counts()

In [ ]:
target = {'BUSINESS': 0, 'SPORTS': 1, 'CRIME': 2, 'SCIENCE': 3}

df_balanced['category_num'] = df_balanced['category'].map({
    'BUSINESS': 0,
    'SPORTS': 1,
    'CRIME': 2,
    'SCIENCE': 3
})

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df_balanced.text,
    df_balanced.category_num,
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df_balanced.category_num
)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
clf = Pipeline([
     ('vectorizer_bow', CountVectorizer(ngram_range = (1, 1))),        #using the ngram_range parameter
     ('Multi NB', MultinomialNB())
])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# TF-IDF

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
df = pd.read_csv("/content/Ecommerce_data.csv")

In [30]:
import numpy as np

In [43]:
print(df.columns)
df['label'].value_counts()
df['target'] = df.label.map({"Household":0,"Electronics":1,"Clothing & Accessories":2,"Books":3})

Index(['Text', 'label'], dtype='object')


In [40]:
# Only Preprocess
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
  doc = nlp(text)
  fil_token = []
  for token in doc:
    if token.is_punct or token.is_stop:
      continue
    else:
      fil_token.append(token.lemma_)
  return " ".join(fil_token)

In [52]:
df.Text = df.Text.apply(lambda x: preprocess(x))

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Text, df.target, test_size = 0.2, stratify=df.label)

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),        #using the ngram_range parameter
     ('Random Forest', RandomForestClassifier())
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1200
           1       0.98      0.97      0.97      1200
           2       0.98      0.98      0.98      1200
           3       0.97      0.97      0.97      1200

    accuracy                           0.97      4800
   macro avg       0.97      0.97      0.97      4800
weighted avg       0.97      0.97      0.97      4800



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),
     ('KNN', KNeighborsClassifier())
])


Seperate

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    "Thor eating pizza, Loki is eating pizza, Ironman ate pizza already",
    "Apple is announcing new iphone tomorrow",
    "Tesla is announcing new model-3 tomorrow",
    "Google is announcing new pixel-6 tomorrow",
    "Microsoft is announcing new surface tomorrow",
    "Amazon is announcing new eco-dot tomorrow",
    "I am eating biryani and you are eating grapes"
]

In [ ]:
v = TfidfVectorizer()
v.fit(corpus)
transform_output = v.transform(corpus)
print(v.vocabulary_)

In [ ]:
#IDF-Calc
all_feature_names = v.get_feature_names_out()
for word in all_feature_names:
    indx = v.vocabulary_.get(word)
    idf_score = v.idf_[indx]
    print(f"{word} : {idf_score}")

In [ ]:
print(transform_output.toarray())

#Spacy: GloVe
Word Embedding

In [ ]:
import pandas as pd
df = pd.read_csv("Fake_Real_Data.csv")
df.columns
df.label.value_counts()

In [63]:
df.label = df.label.map({'Fake':0,'Real':1})

We will use **glove** embeddings from spacy

In [69]:
# Note: we get vector represetation from spacy
import spacy
nlp = spacy.load("en_core_web_sm")
df['fil_text'] = df.Text.apply(lambda x: nlp(x).vector)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.fil_text.values, df.label, test_size = 0.2)

In [70]:
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#vector to be in proper range
scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test, y_pred))

In [ ]:
from  sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')
clf.fit(X_train_2d, y_train)
y_pred = clf.predict(X_test_2d)
print(classification_report(y_test, y_pred))

# Gensim: Word2vec

sentence representation

In [ ]:
#GloVe
def load_embedding_model():
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(wv_from_bin.key_to_index.keys()))
    return wv_from_bin

Word2vec

In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')
wv_great = wv["great"]                          # Vector representation
wv.similarity(w1="great", w2="good")
wv.most_similar("good")
wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=5)
wv.doesnt_match(["facebook", "cat", "google", "microsoft"])

Whole pipeline

In [ ]:
import pandas as pd
df = pd.read_csv("fake_and_real_news.csv")
df['label'].value_counts()
df['label_num'] = df['label'].map({'Fake' : 0, 'Real': 1})
r2 = wv.get_mean_vector(["good", "great"],pre_normalize=False)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg") # if this fails then run "python -m spacy download en_core_web_lg" to download that model
def preprocess_and_vectorize(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return wv.get_mean_vector(filtered_tokens)

In [ ]:
df['vector'] = df['Text'].apply(lambda text: preprocess_and_vectorize(text))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.vector.values,
    df.label_num,
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.label_num
)

In [ ]:
X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
clf = GradientBoostingClassifier()
clf.fit(X_train_2d, y_train)
y_pred = clf.predict(X_test_2d)
print(classification_report(y_test, y_pred))

Predictions

In [ ]:
test_news = [
    "Michigan governor denies misleading U.S. House on Flint water (Reuters) - Michigan Governor Rick Snyder denied Thursday that he had misled a U.S. House of Representatives committee last year over testimony on Flintâ€™s water crisis after lawmakers asked if his testimony had been contradicted by a witness in a court hearing. The House Oversight and Government Reform Committee wrote Snyder earlier Thursday asking him about published reports that one of his aides, Harvey Hollins, testified in a court hearing last week in Michigan that he had notified Snyder of an outbreak of Legionnairesâ€™ disease linked to the Flint water crisis in December 2015, rather than 2016 as Snyder had testified. â€œMy testimony was truthful and I stand by it,â€ Snyder told the committee in a letter, adding that his office has provided tens of thousands of pages of records to the committee and would continue to cooperate fully.  Last week, prosecutors in Michigan said Dr. Eden Wells, the stateâ€™s chief medical executive who already faced lesser charges, would become the sixth current or former official to face involuntary manslaughter charges in connection with the crisis. The charges stem from more than 80 cases of Legionnairesâ€™ disease and at least 12 deaths that were believed to be linked to the water in Flint after the city switched its source from Lake Huron to the Flint River in April 2014. Wells was among six current and former Michigan and Flint officials charged in June. The other five, including Michigan Health and Human Services Director Nick Lyon, were charged at the time with involuntary manslaughter",
    " Sarah Palin Celebrates After White Man Who Pulled Gun On Black Protesters Goes Unpunished (VIDEO) Sarah Palin, one of the nigh-innumerable  deplorables  in Donald Trump s  basket,  almost outdid herself in terms of horribleness on Friday."
]
test_news_vectors = [preprocess_and_vectorize(n) for n in test_news]
clf.predict(test_news_vectors)

# fastText

Defaul: skip-gram

model = fasttext.train_unsupervised('data', 'cbow')


In [ ]:
# model = fasttext.load_model('C:\\Code\\nlp-tutorials\\downloads\\cc.en.300.bin')
# model.get_analogies("berlin","germany","france")
# model.get_word_vector("good")
# model.get_nearest_neighbors("halwa")

##Classification

In [ ]:
import pandas as pd
import re
df= pd.read_csv("ecommerce_dataset.csv", names=["category", "description"], header=None)
df.dropna(inplace=True)
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)
df['category'] = '__label__' + df['category'].astype(str)
df['category_description'] = df['category'] + ' ' + df['description']

In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [ ]:
df['category_description'] = df['category_description'].map(preprocess)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

In [ ]:
import fasttext
model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

In [ ]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")
# output = __label__electronics

In [ ]:
model.get_nearest_neighbors("painting")

# K-Means


In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv("income.csv")
plt.scatter(df.Age,df['Income($)'])

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df[['Income($)']])
df['Income($)'] = scaler.transform(df[['Income($)']])

scaler.fit(df[['Age']])
df['Age'] = scaler.transform(df[['Age']])

In [ ]:
km = KMeans(n_clusters=3)
y_predicted = km.fit_predict(df[['Age','Income($)']])
df['cluster']=y_predicted
km.cluster_centers_



ELBOW Plot

In [ ]:
sse = []
k_rng = range(1,10)
for k in k_rng:
    km = KMeans(n_clusters=k)
    km.fit(df[['Age','Income($)']])
    sse.append(km.inertia_)

#GMM

#GloVe Stanford a1

In [ ]:
from gensim.models import KeyedVectors
import scipy as sp
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import nltk
nltk.download('reuters')
from nltk.corpus import reuters

In [ ]:
START_TOKEN = '<START>'
END_TOKEN = '<END>'
def read_corpus(category="crude"):
    files = reuters.fileids(category)
    return [[START_TOKEN] + [w.lower() for w in list(reuters.words(f))] + [END_TOKEN] for f in files]

In [ ]:
reuters_corpus = read_corpus()
print(reuters_corpus[:3], compact=True, width=100)

In [ ]:
def distinct_words(corpus):
    corpus_words = []
    num_corpus_words = -1
    corpus_words = {word for doc in corpus for word in doc}
    corpus_words = sorted(list(corpus_words))
    num_corpus_words = len(corpus_words)
    return corpus_words, num_corpus_words

In [ ]:
def compute_co_occurrence_matrix(corpus, window_size=4):
    words, num_words = distinct_words(corpus)
    word2Ind = {}
    for index, word in enumerate(words):
        word2Ind[word] = index

    M = np.zeros((num_words, num_words))
    for sentence in corpus:
        for i in range(len(sentence)):
            start = max(0, i - window_size)
            end = min(i + window_size + 1, len(sentence))
            words_in_window = sentence[start:end]
            curr_word = sentence[i]
            for word in words_in_window:
                if word!=curr_word:
                  M[word2Ind[curr_word], word2Ind[word]] += 1

    return M, word2Ind

In [ ]:
def reduce_to_k_dim(M, k=2):
    n_iters = 10     # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    svd = TruncatedSVD(n_components = k, n_iter = n_iters)
    M_reduced = svd.fit_transform(M)
    return M_reduced

In [ ]:
def plot_embeddings(M_reduced, word2ind, words):
    word_idxs = [word2ind[word] for word in words]
    word_vectors = M_reduced[word_idxs]
    x_coords = [vec[0] for vec in word_vectors]
    y_coords = [vec[1] for vec in word_vectors]
    for i, word in enumerate(words):
        x = x_coords[i]
        y = y_coords[i]
        plt.scatter(x, y, marker='x', color='red')
        plt.text(x, y, word, fontsize=9)
    plt.show()

##GloVe

In [ ]:
def load_embedding_model():
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(wv_from_bin.key_to_index.keys()))
    return wv_from_bin

In [ ]:
wv_from_bin = load_embedding_model()

In [ ]:
#Getting word vector matrix
M = []
curInd = 0
for w in words:
    try:
        M.append(wv_from_bin.word_vec(w))
        word2ind[w] = curInd
        curInd += 1
    except KeyError:
        continue


In [ ]:
M, word2ind = get_matrix_of_vectors(wv_from_bin)
M_reduced = reduce_to_k_dim(M, k=2)
M_lengths = np.linalg.norm(M_reduced, axis=1)
M_reduced_normalized = M_reduced / M_lengths[:, np.newaxis] # broadcasting